### Importovanie knižníc a nástrojov

In [ ]:
import matplotlib.pyplot as plt
import pathlib
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

### Zadefinovanie ciest k datasetom

In [ ]:
TRAIN_DATA_DIR = './train_8vs8'
VALIDATION_DATA_DIR = './validation_8vs8'
TEST_DATA_DIR = './test_8vs8'

TRAIN_DATASET_PATH = pathlib.Path(TRAIN_DATA_DIR)
VALIDATION_DATASET_PATH = pathlib.Path(VALIDATION_DATA_DIR)
TEST_DATASET_PATH = pathlib.Path(TEST_DATA_DIR)

### Vytvorenie ImageDataGenerator-ov, ktoré slúžia pre načítanie, predspracovanie a augmentáciu obrázkov

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_validation_datagen = ImageDataGenerator()

### Vytvorenie datasetov pomocou flow_from_directory funkcie ImageDataGenerator-a

In [ ]:
BATCH_SIZE = 64
IMG_SIZE = (240, 240)

train_dataset = train_datagen.flow_from_directory(
    TRAIN_DATASET_PATH, 
    target_size=IMG_SIZE, 
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

validation_dataset = test_validation_datagen.flow_from_directory(
    VALIDATION_DATASET_PATH, 
    target_size=IMG_SIZE, 
    class_mode='categorical', 
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataset = test_validation_datagen.flow_from_directory(
      TEST_DATASET_PATH, 
      target_size=IMG_SIZE, 
      class_mode='categorical', 
      batch_size=BATCH_SIZE, 
      shuffle=False
)

### Vykreslenie 5 augmentovaných trenovacích obrázkov

In [ ]:
plot_dataset = train_datagen.flow_from_directory(
    TRAIN_DATASET_PATH, 
    target_size=IMG_SIZE, 
    class_mode='categorical', 
    batch_size=1,
    shuffle=True
)

fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(50,50))

for i in range(5):
  image = next(plot_dataset)[0].astype('uint8')
  image = np.squeeze(image)

  ax[i].imshow(image)
  ax[i].axis('off')

### Model

*   EfficientNetB1, ktorý je trénovaný pomocou transfer learningu
*   Modelu boli odobraté posledné vrstvy pre klasifikovanie 1000 tried a bola mu pridaná Flatten vrstva a Dense vrstva so softmax aktivačnou funkciou pre klasifikovanie 16 tried húb



In [ ]:
b1 = EfficientNetB1(input_shape=(240, 240, 3), weights='imagenet', include_top=False, classifier_activation="softmax") 

In [ ]:
for layer in b1.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(b1.output)

In [ ]:
prediction = Dense(16, activation='softmax')(x)
model = Model(inputs=b1.input, outputs=prediction)

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(0.0003),
  metrics=['accuracy']
)

In [ ]:
model.summary()

### Zadefinovanie predčasného zastavenia trénovania modelu pomocou EarlyStopping a monitorovania vývoju metrík pomocou TensorBoard

In [ ]:
NAME = "8-vs-8-CNN"

early_stop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

### Trénovanie modelu

In [ ]:
history = model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=20,
  callbacks=[early_stop, tensorboard],
  batch_size=BATCH_SIZE)

### Vykreslenie grafov hodnôt presnosti a stratovej funkcie, ktoré mal model počas trénovania

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Evaluácia vytrénovaného modelu

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset,batch_size=BATCH_SIZE)

In [ ]:
print("Testovacia loss {0:.2f}, Testovacia accuracy {1:.2f}%".format(test_loss, test_accuracy * 100))

In [ ]:
Y_true_classes = test_dataset.classes
Y_predict = model.predict(test_dataset)

Y_pred_classes = np.argmax(Y_predict, axis=1)

CONFUSION_MATRIX = confusion_matrix(Y_true_classes, Y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=CONFUSION_MATRIX)

disp = disp.plot()

In [ ]:
print(classification_report(Y_true_classes, Y_pred_classes))